# <p style="text-align: center;">MIS 285N: Big Data and Distributed Programming</p>
# <p style="text-align: center;">Project - 1 : Apache Spark</p>
## <p style="text-align: center;">Instructor: Dr. Ramesh Yerraballi</p>
## <p style="text-align: center;">Due: Tuesday, September 14th submitted via Canvas by 11:59 pm</p>

Your work should be written in a **Jupyter notebook**.   

Also, please make sure your code runs in your notebook before submitting.

**Note:**

This project is based on Map-Reduce Framework. In these you will get to work with Spark and will get to know how 
does spark work, what functionalities does spark provide, what does map-reduce framework do and why is it useful. 

In this project you will be implementing a basic song recommender system. You will be given a dataset where there are multiple csv files. These csv files have data corresponding to song play count and song information.

The data you will use is provided in a zip file along with this notebook. The __msd.zip__ archive contains:
1. **'kaggle_visible_evaluation_triplets.txt'**. We will be using the visible part of the testing data to understand the working on Apache Spark.  The user's listening history is provided as: (user, song, play count).  
2. In **'kaggle_songs.txt'** file, each song is marked using an index for easier representation of songs.  
3. And **'kaggle_users.txt'** file is the canonical list of user identifiers.
4. Take **'MSDChallengeGettingstarted.pdf'** as your reference.



### **What to turn in?**  

A zip folder which will have:
1. Jupyter Notebook
2. A brief report in PDF format on what features you used for recommendation. And a brief explanation of flow of your code. For example,  what RDD does what or, why it was created.
3. datasets folder with the csv files you are using in your notebook.
4. Notebook should use relative path to the csv files in datasets folder.
5. Name of the zip folder - `<your_name>_<your_partner_name>.zip`

This project consists of 4 questions:  

1. Create an RDD with _kaggle_visible_evaluation_triplets.txt_ and replace the song name with the song index from _kaggle_songs.txt_. Identify the number of songs that do not have any rating. 
2. Generate song ratings based on the song play count as a normalized score between 0 and 1. 
3. Identify the popular song based on this rating and recommend songs to user, given user id based on the algorithm used in Movie recommender system from class. 
4. Using Cosine similarity function, identify pair-wise similarity between each pair of users and generate the top 5 most similar users without an overlap in users. 

The above list is the high-level idea about the questions. 

In [1]:
### Starter code ####
import findspark
from operator import add
findspark.init()
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]").setAppName("Songs")
sc = SparkContext(conf = conf)
#### These lines are to tell jupyter where to find Apache Spark ####

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/13 23:52:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/13 23:52:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/13 23:52:50 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/09/13 23:52:50 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
#sc.stop()

In [3]:
## Read triplet file into RDD
triplet_rdd = sc.textFile(r"kaggle_visible_evaluation_triplets.txt") \
    .map(lambda line: line.split("\t")) 

song_rdd = sc.textFile(r"kaggle_songs.txt").map(lambda line: line.split(" "))
user_rdd = sc.textFile(r"kaggle_users.txt").map(lambda line: line.split(" "))

In [4]:
triplet_rdd.take(2)

[['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SOBONKR12A58A7A7E0', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SOEGIYH12A6D4FC0E3', '1']]

In [5]:
song_rdd.take(2)

[['SOAAADD12AB018A9DD', '1'], ['SOAAADE12A6D4F80CC', '2']]

In [6]:
user_rdd.take(2)

[['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d'],
 ['d7083f5e1d50c264277d624340edaaf3dc16095b']]

## Step 1: 
Replace song name with song index and identify the number of songs without user history

In [7]:
song = {}
songItr = song_rdd.collect()
for s in songItr:
    song [s[0]] = s[1]

In [8]:
triplet_q1_rdd = triplet_rdd.map(lambda x: (x[1], x[0], x[2]))\
                                .map(lambda x: [x[1],song[x[0]],x[2]])
triplet_q1_rdd.take(5)

[['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '25150', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '68212', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '87433', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '123630', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '58821', '1']]

In [9]:
# triplet_songbase = triplet_rdd.map(lambda x : (x[1] ,(x[0], x[2]))).join(song_rdd)
# triplet_songbase.take(2)

In [10]:
triplet_songbase = triplet_rdd.map(lambda x : (x[1] ,(x[0], x[2]))).join(song_rdd)

replaced_rdd = triplet_songbase.map(lambda x: (x[1][0][0], x[1][1], x[1][0][1]))
replaced_rdd.take(5)

[('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '25150', '1'),
 ('c34670d9c1718361feb93068a853cead3c95b76a', '25150', '1'),
 ('c5006d9f41f68ccccbf5ee29212b6af494110c5e', '25150', '1'),
 ('e4332e11f4df6dd26673bb6b085e9a2bbdc9b8a5', '25150', '2'),
 ('baf2fe5885ab93fbbdb7fecc6691788e70afb6c8', '25150', '4')]

In [11]:
hacked_triplet_rdd = triplet_rdd.map(lambda x: (x[1], x[0], x[2]))


In [12]:
hacked_triplet_rdd.take(2)

[('SOBONKR12A58A7A7E0', 'fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '1'),
 ('SOEGIYH12A6D4FC0E3', 'fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '1')]

In [13]:
song_count = hacked_triplet_rdd.map(lambda x : x[0]).collect()
final_count = song_rdd.count()-len(sorted(sc.parallelize(song_count).distinct().collect()))

22/09/13 23:53:02 WARN TaskSetManager: Stage 10 contains a task of very large size (3726 KiB). The maximum recommended task size is 1000 KiB.


In [14]:
print("Total number of songs without user history are", final_count)

Total number of songs without user history are 223007


## Step 2:
Generate song ratings based on the play_count. For example, if (song_1, 5; song_2, 10; song_3, 5) i.e., song_1 is played 5 times, song_2 is played 10 times and song_3 is played 5 times, the normalized rating score should be 0.25, 0.5 and 0.25 respectively. 
Similarly, generate the rating for all the songs. You may notice that based on all songs, the rating is almost always very low. So, think of the best way to convert song count to ratings. (Hint: Try generating ratings based on each user's song play history)

In [15]:
count = replaced_rdd.map(lambda x: (x[1],(int(x[2])))).reduceByKey(add).collect()







In [16]:
ratio = 1 / len(count)

rating = [(x[0],x[1] * ratio) for x in count]


In [17]:
user_rdd.take(5)

[['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d'],
 ['d7083f5e1d50c264277d624340edaaf3dc16095b'],
 ['d68dc6fc25248234590d7668a11e3335534ae4b4'],
 ['9be82340a8b5ef32357fe5af957ccd54736ece95'],
 ['841b2394ae3a9febbd6b06497b4a8ee8eb24b7f8']]

In [18]:
ulist = replaced_rdd.map(lambda x: (x[0],(x[1], x[2]))).groupByKey().mapValues(list)

In [19]:
user_song_map = {}
user_list = ulist.collect()
for x in user_list:
    total = sum( int(cnt) for _,cnt in x[1])
    user_song_map[x[0]] = total
    
    
urating = replaced_rdd.map(lambda x: (x[0],(x[1], int(x[2])/ user_song_map[x[0]])))

base = urating.map(lambda x: (x[1][0], (x[1][1]))).reduceByKey(add).collect()
base_rdd = urating.map(lambda x: (x[1][0], (x[1][1]))).groupByKey().mapValues(len).collect()


In [20]:
rate_map = {}
for i in range(0, len(base)):
    rate_map[base[i][0]] = base[i][1] / base_rdd[i][1]

In [21]:
#for i in range(0, len(rating_map)):
print(rate_map['245936'])

0.0707117928788383


In [22]:
user_song_rdd = triplet_q1_rdd.map(lambda x: (x[0], (x[1], x[2], rate_map[x[1]])))
user_song_rdd.take(5)

[('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d',
  ('25150', '1', 0.15281650173989408)),
 ('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d',
  ('68212', '1', 0.11858674254771402)),
 ('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d',
  ('87433', '1', 0.09207443865735128)),
 ('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d',
  ('123630', '1', 0.1013900129520677)),
 ('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d',
  ('58821', '1', 0.08829468115399224))]

## Step 3: 
For a given user_id (choose one by yourselves), rating, recommend 5 other songs from the list. One way to do this is based on another user who liked the same song liked by this user with rating more than the given rating and recommend the 5 songs based on the matched user's rating. 

In [23]:
import numpy as np

In [24]:
givenSongId = '25150'; 
givenUserId = 'fd50c4007b68a3737fe052d5a4f78ce8aa117f3d';   
avgRating = 0.0;   

In [25]:
ratings_rdd = user_song_rdd.map(lambda x: (x[0], [x[1][0], x[1][2]]))
ratings_rdd.collect()

[('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', ['25150', 0.15281650173989408]),
 ('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', ['68212', 0.11858674254771402]),
 ('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', ['87433', 0.09207443865735128]),
 ('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', ['123630', 0.1013900129520677]),
 ('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', ['58821', 0.08829468115399224]),
 ('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', ['351764', 0.09948196898852323]),
 ('d7083f5e1d50c264277d624340edaaf3dc16095b', ['315822', 0.10181288790469191]),
 ('d7083f5e1d50c264277d624340edaaf3dc16095b', ['312787', 0.02670564576480902]),
 ('d7083f5e1d50c264277d624340edaaf3dc16095b', ['136848', 0.07976636935468889]),
 ('d7083f5e1d50c264277d624340edaaf3dc16095b', ['67457', 0.11764705882352941]),
 ('d7083f5e1d50c264277d624340edaaf3dc16095b', ['11841', 0.10843781104641251]),
 ('d7083f5e1d50c264277d624340edaaf3dc16095b', ['381063', 0.07783252254981597]),
 ('d7083f5e1d50c264277d624340edaaf3dc16095b', [

In [26]:

def liked(row):
    if((row[1][0] == givenSongId) and (float(row[1][1]) >= avgRating)):
        return True
    return False


users = ratings_rdd.filter(liked) \
                    .map(lambda line: [line[0], line[1][1]] )
#users.take(5)
users.collect()

[['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 0.15281650173989408],
 ['c34670d9c1718361feb93068a853cead3c95b76a', 0.15281650173989408],
 ['c5006d9f41f68ccccbf5ee29212b6af494110c5e', 0.15281650173989408],
 ['e4332e11f4df6dd26673bb6b085e9a2bbdc9b8a5', 0.15281650173989408],
 ['baf2fe5885ab93fbbdb7fecc6691788e70afb6c8', 0.15281650173989408],
 ['f6e34f0a68d5ea1344511e33486f956de361db78', 0.15281650173989408],
 ['e326c4b9fe3659ec1dc3af53fd7e0893809dafbc', 0.15281650173989408],
 ['00f7c493ee64884998ea98d9f5bed87bc4a0afcf', 0.15281650173989408],
 ['daa9e7e53ae787ab4f1b5518b695198947d821a2', 0.15281650173989408],
 ['cd4321d8fd42ba44996e7f34c2f6404cf5884696', 0.15281650173989408],
 ['bcb1e6d620cf522390d5c92bae26936928e0b588', 0.15281650173989408],
 ['3d28488a659d1cacc598d17074b233985271c65e', 0.15281650173989408],
 ['eead1f1e3ad91575346f9ce826ddaea19bff80ed', 0.15281650173989408],
 ['07260aa795b86382fbddea37b7b8f6f7d5a1db0a', 0.15281650173989408],
 ['4fbde29be137e7179046c148f9e15db17c3278be', 0.

In [27]:
songTypes = song_rdd.map(lambda x: (x[1], [x[0]]))
songTypes.take(5)

[('1', ['SOAAADD12AB018A9DD']),
 ('2', ['SOAAADE12A6D4F80CC']),
 ('3', ['SOAAADF12A8C13DF62']),
 ('4', ['SOAAADZ12A8C1334FB']),
 ('5', ['SOAAAFI12A6D4F9C66'])]

In [28]:
usersSongs = ratings_rdd.join(users) \
                .map(lambda line: (line[1][0][0], [line[0], line[1][0][1]]))

usersSongs.collect()

[('215410', ['baf2fe5885ab93fbbdb7fecc6691788e70afb6c8', 0.07215342340439954]),
 ('304533',
  ['baf2fe5885ab93fbbdb7fecc6691788e70afb6c8', 0.024445238619693447]),
 ('310063', ['baf2fe5885ab93fbbdb7fecc6691788e70afb6c8', 0.064949148581721]),
 ('196055', ['baf2fe5885ab93fbbdb7fecc6691788e70afb6c8', 0.06653213733045665]),
 ('25150', ['baf2fe5885ab93fbbdb7fecc6691788e70afb6c8', 0.15281650173989408]),
 ('98901', ['baf2fe5885ab93fbbdb7fecc6691788e70afb6c8', 0.0780862710110136]),
 ('290345', ['baf2fe5885ab93fbbdb7fecc6691788e70afb6c8', 0.09297414977886437]),
 ('231280', ['baf2fe5885ab93fbbdb7fecc6691788e70afb6c8', 0.06654120773349081]),
 ('55973', ['baf2fe5885ab93fbbdb7fecc6691788e70afb6c8', 0.1250548076015366]),
 ('156952', ['baf2fe5885ab93fbbdb7fecc6691788e70afb6c8', 0.03982683982683983]),
 ('83120', ['baf2fe5885ab93fbbdb7fecc6691788e70afb6c8', 0.054764232183587024]),
 ('64242', ['cd4321d8fd42ba44996e7f34c2f6404cf5884696', 0.09602136958129683]),
 ('381376', ['cd4321d8fd42ba44996e7f34c2f6404

In [29]:
def notTheGivenSong(row):
    if(row[0] != givenSongId):
        return True
    return False

usersSongTypes = usersSongs.join(songTypes) \
                                .map(lambda x: (x[0], x[1][0][1])) \
                                .filter(notTheGivenSong) \
                                .reduceByKey(lambda accum, n: float(accum) + float(n)) 

usersSongTypes.collect()

[('64379', 0.07850178901787351),
 ('183832', 0.26058232555084854),
 ('75475', 0.09395834978607352),
 ('87433', 43.18291173029761),
 ('170536', 0.1849067761206284),
 ('170520', 0.07689406546575664),
 ('123630', 35.0809444814152),
 ('306349', 0.09232381704868867),
 ('13514', 0.3474651081658299),
 ('79511', 0.12373854391926681),
 ('261596', 12.774459024308124),
 ('96461', 0.12331683910631279),
 ('359476', 6.747602451704205),
 ('305804', 4.204612708197906),
 ('343599', 0.5134288817789536),
 ('163940', 0.034482758620689655),
 ('312714', 0.06945735436474565),
 ('214624', 0.16863434482084447),
 ('221645', 0.06422413793103449),
 ('268700', 0.047514416083932365),
 ('270428', 0.15804689932552357),
 ('46745', 0.07934816462736373),
 ('6977', 0.5770533756337944),
 ('73369', 1.8793602868644552),
 ('105694', 4.0977159806456145),
 ('363476', 0.03775870433530565),
 ('181039', 0.13250041659133865),
 ('112529', 0.06206698626487368),
 ('119689', 0.1946571637061119),
 ('341295', 0.066152288600405),
 ('7034

In [30]:
recommended_songs = songTypes.join(usersSongTypes) \
                                .map(lambda x: (x[1][0][0], float(x[1][1]))) \
                                .takeOrdered(10, key=lambda x: -x[1])

for i in range(0, 5):
    print(recommended_songs[i][0])
    


SOAUWYT12A81C206F1
SOSXLTC12AF72A7F54
SOEGIYH12A6D4FC0E3
SOVDSJC12A58A7A271
SOFRQTD12A81C233C0


## Step 4: 
1. Compute cosine similarity between all pairs of users. 
2. Sort the similarity score and print the top-5 similar users. 
3. If the top-5 user set has an user appearing more than once, ignore that pair and take the next best pair from the sorted list. 
4. For a given user_id, identify the top-5 similar users and hence song recommendations from other user's list. 

In [31]:
def cosine_similarity(x, y):
  val_s = set([i[0] for i in x]).intersection(set([i[0] for i in y]))
  if len(val_s) == 0:
    return 0
  x_dict, y_dict = {i[0]: i[1] for i in x}, {i[0]: i[1] for i in y}
  num = 0
  for xk in x_dict.keys():
    if xk in y_dict:
      num += x_dict[xk]*y_dict[xk]
  den = sum([abs(i) for i in x_dict.values()])*sum([abs(i) for i in y_dict.values()])
  return num / den

In [32]:
replaced_rdd.take(5)

[('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '25150', '1'),
 ('c34670d9c1718361feb93068a853cead3c95b76a', '25150', '1'),
 ('c5006d9f41f68ccccbf5ee29212b6af494110c5e', '25150', '1'),
 ('e4332e11f4df6dd26673bb6b085e9a2bbdc9b8a5', '25150', '2'),
 ('baf2fe5885ab93fbbdb7fecc6691788e70afb6c8', '25150', '4')]

In [33]:
user_song_list = replaced_rdd.map(lambda line: [line[0], (int(line[2]))])\
                             .reduceByKey(lambda x,y: x+y)\
                            .join(replaced_rdd.map(lambda x: [x[0], (int(x[1]), int(x[2]))]))  
        

random_user = sc.parallelize(user_song_list.take(5000))



In [34]:
grouped_user = random_user.map(lambda line: (line[0], line[1][1]))\
                            .groupByKey()


In [35]:
symmetric = grouped_user.cartesian(grouped_user).filter(lambda line: line[0][0] != line[1][0])


In [36]:
cosines = symmetric.map(lambda line: (line[0][0], line[1][0], cosine_similarity(line[0][1], line[1][1])))

cosines.take(5)

22/09/13 23:53:43 ERROR ShuffleBlockFetcherIterator: Error occurred while fetching local blocks, null


[('84626fa9e6af08a66795dfc89e1e54460b4d8206',
  'd15144794b772b5a4c5cc23b9f0060fb160a2ac9',
  0.002278481012658228),
 ('84626fa9e6af08a66795dfc89e1e54460b4d8206',
  '4fa34280313a14f14862a80ec0c79efd384e1a17',
  0.0010526315789473684),
 ('84626fa9e6af08a66795dfc89e1e54460b4d8206',
  '3da15863fd6d6e0a91674fe893929238636dde83',
  0.0004878048780487805),
 ('84626fa9e6af08a66795dfc89e1e54460b4d8206',
  'e48ec63ed90a119eafe5338b18850ad2502e6910',
  0.0002867383512544803),
 ('84626fa9e6af08a66795dfc89e1e54460b4d8206',
  '2018338f4a23969a9517c59f8ceadee81c77eedd',
  0.0002666666666666667)]

In [37]:
users = cosines.filter(lambda line: line[0] != "84626fa9e6af08a66795dfc89e1e54460b4d8206")
final = users.map(lambda x: x[0],x[1])
final.take(5)

['d15144794b772b5a4c5cc23b9f0060fb160a2ac9',
 '4fa34280313a14f14862a80ec0c79efd384e1a17',
 'd15144794b772b5a4c5cc23b9f0060fb160a2ac9',
 '4fa34280313a14f14862a80ec0c79efd384e1a17',
 'd15144794b772b5a4c5cc23b9f0060fb160a2ac9']